## Otimização de hiperparâmetros



## Introdução



Escolheu-se "árvores aleatórias" como algoritmo para otimizar três hiperparâmetros utilizando-se de redes neurais para achar o melhor conjunto de hiperparâmetros.


## Objetivo



**Objetivo**: use algoritmos genéticos para encontrar um bom conjunto de hiperparâmetros em um experimento de aprendizado de máquina. Escolha um algoritmo que tenha pelo menos 3 hiperparâmetros para serem otimizados.



## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [1]:
import random
import time
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from operator import itemgetter
from funcoes import populacao_inicial_hiperparametros as cria_populacao_inicial
from funcoes import funcao_objetivo_pop_hiperparametros
from funcoes import funcao_objetivo_hiperparametros
from funcoes import funcao_objetivo_standard
from funcoes import selecao_torneio_min_hiperparametros as funcao_selecao
from funcoes import reproducao_shuffle_ordenado_hiperparametros as funcao_cruzamento
from funcoes import mutacao_hiperparametros
from funcoes import mutacao_tendencia_as_cegas
from funcoes import hiperparametros_modelo

## Códigos e discussão



O notebook GA.08 é um dos experimentos da "Lista de experimentos - hp tuning" e aborda o problema de otimização de hiperparâmetros. Desse modo, foi definido um conjunto de hiperparâmetros aos quais quer se investigar possibilidades de um MSE razoável por meio de algoritmos genéticos. Existem funções particularmente interessantes, como é o caso da "funcao_objetivo_hiperparametros", que utiliza do MSE como parâmetro base para determinar o fitness de cada indivíduo da população, naturalmente aplicando uma "punição" àqueles que não cumprem a condição de ser menor que o "MSE_standard_calculated". Outra interessante é a "mutacao_tendencia_as_cegas", que muta um individuo em um gene aleatório, baseado em um gene (aleatório) de um dos três melhores indivíduos. Ainda, é interessante citar a "reproducao_suffle_ordenado", que cria um indivíduo a partir de outros três - "pai", "mae" e "pae", em que cada um deles fornece um gene aleatório para gerar outros três indivíduos, respeitando a ordem estabelecida. Em última instância, é possível explicitar a "mutacao_hiperparametros", que seleciona um gene aleatório do dicionário que contém os valores dos hiperparâmetros e substitui um gene do indivíduo. Sabe-se que "MSE_standard"/"MSE_standard_calculated" é o valor obtido por meio da modificação mínima (apenas da random_seed) dos hiperparâmetros padrões do scikit-learn. Nota-se que foi utilizado o dataframe "diamonds" (que é completo, bem comportado e ideal) do seaborn pois o foco deste notebook é a estruturação do código de algoritmos genéticos. Entretanto, é possível que, com algumas modificações e otimizações, o código possa ser utilizado para dataframes menos ideais.

In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 9
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
CHANCE_MUTACAO_2 = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3
NUM_GERACOES = 5
NUM_GENES = 5
CONJUNTOS_HIPERPARAMETROS = 3
CORTE = 3

# relacionadas ao problema a ser resolvido - dataset
TAMANHO_TESTE = 0.25
SEMENTE_ALEATORIA = 1024
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]
HIPER_RANGE_1=20
HIPER_RANGE_2=10

In [3]:
#define o dataset e os datasets de treino e teste
#função global pois é o modelo que será usado para esta situação. Isso evita que o mesmo código rode várias vezes.
df = sns.load_dataset(DATASET_NAME) #importar dataset

indices = df.index
indices_treino, indices_teste = train_test_split(
indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA) #separar os indices entre dataset de treino e teste

df_treino = df.loc[indices_treino] #separa o dataset entre treino e teste
df_teste = df.loc[indices_teste]

# observe que usamos o .values aqui pois queremos apenas os valores
X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values
X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values
    


In [4]:
# funções locais
def funcao_objetivo_pop(X_treino,X_teste,y_treino,y_teste,SEMENTE_ALEATORIA,populacao,MSE_standard):
    return funcao_objetivo_pop_hiperparametros(X_treino,X_teste,y_treino,y_teste,SEMENTE_ALEATORIA,populacao,MSE_standard_calculated)

def funcao_mutacao(individuo,hiper_range_1,hiper_range_2):
    return mutacao_hiperparametros(individuo,hiper_range_1,hiper_range_2)

def funcao_mutacao_2(individuo, populacao, corte=3):
    return mutacao_tendencia_as_cegas(individuo, populacao, corte=3)

In [5]:
start = time.time() #inicia contagem de tempo
MSE_standard_calculated = funcao_objetivo_standard(X_treino,X_teste,y_treino,y_teste,SEMENTE_ALEATORIA)
populacao = cria_populacao_inicial(TAMANHO_POP, HIPER_RANGE_1,HIPER_RANGE_2)
lista_hiperparametros = []
lista_fitness = []

while len(lista_hiperparametros) != CONJUNTOS_HIPERPARAMETROS:
    
    # Seleção
    fitness = funcao_objetivo_pop(X_treino,X_teste,y_treino,y_teste,SEMENTE_ALEATORIA,populacao,MSE_standard_calculated)
    populacao, fitness_populacao = funcao_selecao(populacao, fitness)
    #print(populacao,type(fitness_populacao),fitness_populacao, type(fitness_populacao))
    
    # Mutação_2
    for k in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO_2:
            individuo = populacao[k]
            populacao[k] = funcao_mutacao_2(individuo,populacao,CORTE)    
    
    # Cruzamento
    pais = populacao[0::3] #alteração do passo para 3 por causa da presença do "pae"
    maes = populacao[1::3]
    paes = populacao[2::3]
    
    
    contador = 0
    
    for pai, mae, pae in zip(pais, maes, paes):
        if random.random() <= CHANCE_CRUZAMENTO: #or pai, mae, pae == pais[0], maes[0], paes[0]:
            filho_1, filho_2, filho_3 = funcao_cruzamento(pai, mae, pae)
            populacao[contador] = filho_1
            populacao[contador + 1] = filho_2
            populacao[contador + 2] = filho_3
        
        contador = contador + 3   
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo,HIPER_RANGE_1,HIPER_RANGE_2)           
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(X_treino,X_teste,y_treino,y_teste,SEMENTE_ALEATORIA,populacao,MSE_standard_calculated)
    #print(fitness)
    menor_fitness = min(fitness)
    #print(menor_fitness)
    posicao = fitness.index(menor_fitness)
    melhor_individuo_ja_visto = (populacao[posicao]) #list is unhasheable
    #print(melhor_individuo_ja_visto)
    if melhor_individuo_ja_visto not in lista_hiperparametros:
        lista_hiperparametros.append(melhor_individuo_ja_visto)
    #print(set_hiperparametros)
        lista_fitness.append(menor_fitness)
    #print(set_fitness)

print()
end = time.time() #finaliza a contagem de tempo
print(f'Aqui estão {CONJUNTOS_HIPERPARAMETROS} conjuntos de hiperparametros decentes:')
print(lista_hiperparametros, lista_fitness)
print(f'por comparação, aqui está o MSE padrão: {MSE_standard_calculated}')
print(f'o tempo de execução do código foi foi de: {(end-start)/60} minutos')


Aqui estão 3 conjuntos de hiperparametros decentes:
[[8, 'poisson', 3], [7, 'poisson', 6], [8, 'friedman_mse', 2]] [2044957.0225911227, 2041663.6045682563, 2040833.5487337704]
por comparação, aqui está o MSE padrão: 3583955.5884534116
o tempo de execução do código foi foi de: 17.14417943159739 minutos


## Conclusão



Este código tem o objetivo de resolver o problema de otimização de hiperparâmetros com algoritmos genéticos. O problema se baseia em encontrar um conjunto de hiperparâmetros razoáveis por meio de algoritmos genéticos, com base em um dicionário que contém os hiperparâmetros e seus valores estabelecidos. O resultado obtido foi satisfatório, pois em geral (dado o fato de que é um algoritmo probabilístico, não se pode afirmar 100% das vezes) o código entrega conjuntos de hiperparâmetros razoáveis (menores que o MSE_standard). Nota-se que foram escolhidos 3 conjuntos com o fim de explicitar que existem várias possibilidades que geram resultados próximos, não se limitando a apenas uma possibilidade. Um problema deste código é que ele demora consideravelmente para terminar de rodar. Naturalmente, uma mínima otimização foi feita, especificamente onde fosse mais óbvio, no entento isso não solucionou o problema. Nesse caso, as ações possíveis a serem tomadas para otimização é usar o módulo DEAP sem o eaSimple (o que permite maior liberadade para modificações, enquanto se pode aproveitar da otimização e rapidez do DEAP) e analisar meticulosamente as funções para evitar repetições.

## Playground



In [6]:
a = 'aaaaa'
for i in a:
    print(i)

a
a
a
a
a


In [7]:
#ideias
#seria uma otima ideia, não implementarei pois achei, até certo grau, reduntante.

#list comprehension inside a dict comprehension

#{
#c:[i for i in range(1,v)] for c,v in hiperparametros.items() #c: chave; v: valor
#}


#**args, *args, **kwargs
######################################

Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.



In [8]:
funcao_objetivo_hiperparametros([9, 'poisson', 7],MSE_standard,punishment=1e6)

NameError: name 'MSE_standard' is not defined

In [ ]:
individuo = [1,2,3]
lst = [[1,2,3],[5,6,7]]

if individuo not in lst:
    print('goofy aah')
else:
    print('yee')

In [ ]:
dicio = {
    'a':10,
    'b':11,
    'c':14,
}
#dicio_2 = (a = 10, b = 11, c = 14)
def func(a,b,c, **kwargs):
    print(a)

In [ ]:
print(tuple([[1],['a'],[2]]))

In [ ]:
dicio = {
    'a':10,
    'b':11,
    'c':14,
}
#dicio_2 = (a = 10, b = 11, c = 14)
def func(a,b,c):
    print(a)

In [ ]:
def tendencia_as_cegas(individuo, populacao, fitness_populacao, corte=3):
    
    print(individuo)
    lista_index = [i for i in range(0, (len(individuo)))]
    
    par_populacao_fitness = list(zip(populacao, fitness_populacao))
    par_populacao_fitness_sorted = sorted(par_populacao_fitness, key=itemgetter(-1))
    
    melhores_individuos_zip = par_populacao_fitness_sorted[:corte]
    
    individuo_melhor = random.choice(melhores_individuos_zip)[0]
    
    print('individuo',individuo_melhor)
    numero_mutacoes = random.randint(1, len(individuo))
    print(numero_mutacoes)
    mutate_random_index = random.sample(lista_index, numero_mutacoes)
    print(mutate_random_index)
    
    if individuo_melhor == individuo or numero_mutacoes == len(individuo):
        return individuo
    else:
        for gene_index in mutate_random_index:
            individuo[gene_index] = individuo_melhor[gene_index]
        
    return individuo

In [ ]:
def gene_hiperparametros(max_depth_range,min_samples_split_range):
    '''Gera um valor aleatório para cada hiperparâmetro em uma dada faixa.
    
    Returns:
        Um valor válido para os 3 hiparparâmetros variados
        '''

    individuo = []
    hiperparametros = [
    [max_depth for max_depth in range(1,max_depth_range)], #max_depth
    ["gini","entropy","log_loss"], #criterion
    [min_samples_split for min_samples_split in range(2,min_samples_split_range)]
                        ]
    
    for i in (range(0,len(hiperparametros))):
        gene = random.sample(hiperparametros[i], k=1)
        individuo.append(gene)
        
    return individuo
        

In [ ]:
def mutacao_hp(individuo,hiper_range_1,hiper_range_2):
    mutacao_individual = 0.25
    hiperparametros_possiveis = hiperparametros_modelo(hiper_range_1,hiper_range_2)
    for i in range(0,len(individuo)):
        if random.random() <= mutacao_individual:
            #mutate_random_hp = random.choice(range(0,len(hiperparametros_possiveis)))
            mutate_random_hp_parameter = random.choice((list(hiperparametros_possiveis.values())[i]))
    
            individuo[i] = mutate_random_hp_parameter
    
    return individuo

In [ ]:
def funcao_objetivo_pop_hiperparametros(populacao):

    #print(populacao)
    fitness_pop = []
    high_fit =[] #fit superior a baseline
    low_fit = [] #fit inferior a baseline
    for individuo in populacao:
        populacao_MSE_fit = funcao_objetivo_hiperparametros(individuo)
        individuo = individuo.extend(populacao_MSE_fit)
    
    for individuo_MSE_fit in populacao_MSE_fit:
        if individuo_MSE_fit[-1] != individuo_MSE_fit[-2]: # index([-2,-1]) = [MSE, fitness]
            high_fit.append(individuo_MSE_fit)
            
        else:
            low_fit.append(individuo_MSE_fit)
            
    low_fit_sorted = sorted(low_fit, key=itemgetter(-1)) #itemgetter muito util para sort() em listas de listas
    for low_fit_individuo in low_fit_sorted:
        print(low_fit_sorted)
        low_fit_individuo[1] = low_fit.index(low_fit_individuo)*500
        
        
    return low_fit + high_fit
#ideia: detectar aqueles com o fit menor do que o standard, organiza-los em ordem crescente, de modo que index*500 == fitness